# Casador de Impedâncias

Caracterizamos uma série de casadores de imepedância de toco simples para frequências ao redor de 125 MHz.

Nesse caso os dados foram salvos direto do analisador de espectro.
Apesar do número de pontos muito maior, a forma de analisar é a mesma.

**Objetivo**: apresentar os dados coletados de forma informativa.

In [ ]:
# Import necessary modules
import numpy
from matplotlib import pyplot

# Only in Jupyter Notebooks
%matplotlib notebook

O analisador de espectro salva toda sua configuração como cabeçalho do arquivo, por isso é importante armazenar esses dados caso deseje-se reproduzir os experimentos.

In [ ]:
%cat casador/50ohm.csv

Novamente usamos `numpy.loadtxt`, mas agora o cabeçalho tem 44 linhas.

In [ ]:
data = numpy.loadtxt('casador/50ohm.csv', delimiter=',', skiprows=44)
data.shape

Antes de medir os casadores, verificamos as medidas sobre uma carga casada (50 Ω), a que desejamos casar (150 Ω nominais), um aberto e um curto.

In [ ]:
fig, ax = pyplot.subplots(1, 1)

for fname in ['50ohm', '150ohm', 'open', 'short']:
    data = numpy.loadtxt('casador/{}.csv'.format(fname),
                         delimiter=',', skiprows=44)
    ax.plot(data[:, 0] * 1e-6, data[:, 1])

ax.legend([r'$50\,\Omega$', r'$150\,\Omega$', 'Open', 'Short'])
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Power (dBm)')
ax.grid()

Verificamos que o curto e o aberto apresentam aproximadamente o mesmo sinal, como esperado pela teoria.
Sabemos que esses valores deveriam representar reflexão total, portanto podemos usá-los como referência para descontar as perdas nos cabos e isoladores utilizados na montagem experimental.

Como as curvas são bastante planas, usaremos o valor médio de uma delas como referência.

Alternativamente poderíamos usar o vetor completo, com um valor de referência para cada frequência, mas neste caso o resultado seria muito similar.

In [ ]:
short = numpy.loadtxt('casador/short.csv', delimiter=',',
                      skiprows=44, usecols=1)
short.shape

In [ ]:
reference = short.mean()
reference

Usamos 2 *loops* para carregar os sinais medidos para cada configuração de casador.

Os valores de `d` e `l` nos nomes dos arquivos indicam os comprimentos (em centímetros) dos cabos coaxiais utilizados como separador da carga e toco em curto, respectivamente.

In [ ]:
fig, ax = pyplot.subplots(1, 1)

for d in range(23, 28):
    for l in range(16, 21):
        data = numpy.loadtxt('casador/d{}-l{}.csv'.format(d, l),
                             delimiter=',', skiprows=44)
        ax.plot(data[:, 0] * 1e-6, data[:, 1] - reference,
                label='d = {} cm, l = {} cm'.format(d, l))

ax.legend()
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Reflection (dB)')
ax.grid()

Mesmo que a legenda não fosse mostrada, o gráfico está muito poluído e não permite que se conclua nada sobre o experimento.

Considerando que temos 3 variáveis independentes (`d`, `l` e a frequência), precisamos considerar o que buscamos medir para determinar a melhor maneira de visualizar os dados.

Primeiramente carregamos os dados de forma a evidenciar os 3 eixos de variveis independentes (isso só funciona porque todos os dados apresentam os mesmos valores de amostragem em frequência).

In [ ]:
d = numpy.arange(23, 28)
l = numpy.arange(16, 21)
for dd in d:
    for ll in l:
        if dd == 23 and ll == 16:
            data = numpy.loadtxt('casador/d{}-l{}.csv'.format(dd, ll),
                             delimiter=',', skiprows=44)
            freq = data[:, 0] * 1e-6
            refl = numpy.zeros((len(freq), 5, 5)) - reference
            refl[:, 0, 0] += data[:, 1]
        else:
            refl[:, ll - 16, dd - 23] += numpy.loadtxt(
                'casador/d{}-l{}.csv'.format(dd, ll),
                delimiter=',', skiprows=44, usecols=1)

print(freq.shape, l.shape, d.shape, refl.shape)

Vamos inicialmente analisar como os casadores se comportam se nosso objetivo fosse melhorar o casamento em 120 MHz.

Faremos um corte nos dado no eixo da frequência.
Para tanto precisamos determinar o índice da frequência desejada (ou o mais próximo possível).

In [ ]:
i = numpy.argmin(numpy.abs(freq - 120))
i, freq[i]

In [ ]:
fig, ax = pyplot.subplots(1, 1)

ax.imshow(refl[i, :, :], origin='lower')

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.imshow(refl[i, :, :], origin='lower', cmap='magma',
                extent=(d[0]-0.5, d[-1]+0.5, l[0]-0.5, l[-1]+0.5))
pyplot.colorbar(cax, label='Reflection (dB)')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

Uma alternativa ainda mais quantitativa é usar contornos, mas cuidado com artefatos.

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.contourf(d, l, refl[i, :, :])
pyplot.colorbar(cax, label='Reflection (dB)')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.contour(d, l, refl[i, :, :])
ax.clabel(cax, fmt='%.1f', fontsize='small')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

E se buscássemos o mínimo de reflexão para cada configuração de casador sem nos importar com a sua frequência central?

Por exemplo, para o casado com d = 23 cm e l = 16 cm, obteríamos:

In [ ]:
refl[:, 0, 0].min()

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.contourf(d, l, refl.min(axis=0))
pyplot.colorbar(cax, label='Reflection (dB)')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

Por fim, queremos agora saber em que freqências cada um dos mínimos ocorre.

In [ ]:
refl.argmin(axis=0)

In [ ]:
refl[841, 0, 0] == refl[:, 0, 0].min()

In [ ]:
freq[refl.argmin(axis=0)]

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.contour(d, l, freq[refl.argmin(axis=0)])
ax.clabel(cax, fmt='%.0f MHz', fontsize='small')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

## Exercício

Sobreponha os gráficos de mínimo de reflexão e frequência onde ocorre o mínimo de forma a apresentar ambas as informções simultaneamente.

### Bônus

Formate o gráfico de modo a possibilitar a mesma compreensão caso ele seja impresso em preto-e-branco.

## Curiosidade: Modelo Teórico

In [ ]:
f, l, d = numpy.mgrid[100:140:81j, 16:21:121j, 23:28:121j]
lda = 2.0e4 / f

zL = (134 - 40j) / 50.0
gL = (zL - 1) / (zL + 1)
gd = gL * numpy.exp(-numpy.pi*4j*d/lda)
zd = (1 + gd) / (1 - gd)

zs = 0
gs = (zs - 1) / (zs + 1)
gl = gs * numpy.exp(-numpy.pi*4j*l/lda)
zl = (1 + gl) / (1 - gl)

zin = 1 / (1/zd + 1/zl)
gin = (zin - 1) / (zin + 1)
refl = 20*numpy.log10(numpy.abs(gin))

fig, ax = pyplot.subplots(1, 1)

cax = ax.contourf(d[0,0,:], l[0,:,0], refl.min(axis=0), cmap='gray',
                 levels=numpy.arange(-40, -19, 2.5), extend='min')
pyplot.colorbar(cax, label='Reflection (dB)')

cax = ax.contour(d[0,0,:], l[0,:,0], f[:,0,0][refl.argmin(axis=0)],
                 cmap='winter')
ax.clabel(cax, fmt='%.0f MHz', fontsize='small')

ax.set_xlabel(r"$d$ (cm)")
ax.set_ylabel(r"$\ell$ (cm)")

ax.set_aspect(1)